#################################################
### ** Created by Marcelo Oliveira on 24/06/2021. **
### **Disciplina de Fundametos de IA aplicados ao Diagnóstico de Doenças.**
### ** Copyright (c)2021 Instituto de Computação **
### ** Universidade Federal de Alagoas - UFAL. All rights reserved.**
#################################################

References:

Pacotes: FastAI, pydicom, kornia, skimage, opencv-python (cv2), seaborn

In [1]:
!pip install fastai --upgrade

Instalacao dos pacotes:
GDCM: manipulacao do DICOM;
FMI: Funcionalidades adicionais para manipular imagens medicas

In [4]:
!pip install python-gdcm pydicom

In [5]:
pip install kornia

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install scikit-image

In [7]:
import os
import shutil
import cv2
#from pydicom.pixel_data_handlers.util import apply_voi_lut
#import gdcm
#from torchvision.utils import save_image

import fastai; print(fastai.__version__)
import fastcore; print(fastcore.__version__)
import pydicom,kornia,skimage


from fastai.vision.all import *
from fastai.text.all import *
from fastai.collab import *
from fastai.tabular.all import *
from fastai.medical.imaging import *


2.5.3
1.3.26


/Users/marcelo/anaconda3/envs/DataScience/lib/python3.9/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [8]:
#Load DataSet

dataSetPath = Path('/Users/marcelo/IADD/siim-covid19-detection')
print(os.listdir(dataSetPath))

['.DS_Store', 'sample_submission.csv', 'test', 'train', 'train_image_level.csv', 'train_study_level.csv']


In [9]:
train_images = get_dicom_files(f'{dataSetPath}/train')
test_images  = get_dicom_files(f'{dataSetPath}/test')


In [7]:
#dcm =  train_images[0]
#dcm.dcmread()

**DataFrame Creation**
['train_image_level.csv', 'train_study_level.csv']
Load the train_image_level and train_study_level csv files

In [10]:
train_image_level = pd.read_csv(f'{dataSetPath}/train_image_level.csv')
train_study_level = pd.read_csv(f'{dataSetPath}/train_study_level.csv')

In [9]:
#train_image_level.head()

In [11]:
train_image_level['id'] = train_image_level['id'].apply(lambda x: f'{x[:12]}'+ ".dcm")
train_image_level.head()

,id,boxes,label,StudyInstanceUID
0,000a312787f2.dcm,"[{'x': 789.28836, 'y': 582.43035, 'width': 1026.65662, 'height': 1917.30292}, {'x': 2245.91208, 'y': 591.20528, 'width': 1094.66162, 'height': 1761.54944}]",opacity 1 789.28836 582.43035 1815.94498 2499.73327 opacity 1 2245.91208 591.20528 3340.5737 2352.75472,5776db0cec75
1,000c3a3f293f.dcm,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc.dcm,"[{'x': 677.42216, 'y': 197.97662, 'width': 867.79767, 'height': 999.78214}, {'x': 1792.69064, 'y': 402.5525, 'width': 617.02734, 'height': 1204.358}]",opacity 1 677.42216 197.97662 1545.21983 1197.75876 opacity 1 1792.69064 402.5525 2409.71798 1606.9105,9d514ce429a7
3,001398f4ff4f.dcm,"[{'x': 2729, 'y': 2181.33331, 'width': 948.00012, 'height': 604}]",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891.dcm,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'height': 1106}, {'x': 2578.56661, 'y': 998.66667, 'width': 662.66667, 'height': 1120}]",opacity 1 623.23328 1050 1337.23328 2156 opacity 1 2578.56661 998.66667 3241.23328 2118.66667,dfd9fdd85a3e


In [12]:
train_study_level.head() 

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1


Vamos usar o 'StudyInstanceUID' para "JUNTAR" o train_study_level + train_image_level, o que queremos é pegar o filename das imagens para organizar

1 Passo é renomear o nome da coluna id para StudyInstanceUID

In [13]:
train_study_level.rename(columns = {'id':'StudyInstanceUID'}, inplace=True)
train_study_level.head() 

,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1


2 Passo: Remover o "_study"

In [16]:
train_study_level['StudyInstanceUID'] = train_study_level['StudyInstanceUID'].apply(lambda x: f'{x[:12]}')
train_study_level.head() 

,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852,0,1,0,0
1,000c9c05fd14,0,0,0,1
2,00292f8c37bd,1,0,0,0
3,005057b3f880,1,0,0,0
4,0051d9b12e72,0,0,0,1


"Juntando" o train_image_level e train_study_level files

In [17]:
merged = pd.merge(train_image_level, train_study_level, on='StudyInstanceUID')
merged.head()
#merged[20:30]

,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2.dcm,"[{'x': 789.28836, 'y': 582.43035, 'width': 1026.65662, 'height': 1917.30292}, {'x': 2245.91208, 'y': 591.20528, 'width': 1094.66162, 'height': 1761.54944}]",opacity 1 789.28836 582.43035 1815.94498 2499.73327 opacity 1 2245.91208 591.20528 3340.5737 2352.75472,5776db0cec75,0,1,0,0
1,000c3a3f293f.dcm,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,0,0
2,0012ff7358bc.dcm,"[{'x': 677.42216, 'y': 197.97662, 'width': 867.79767, 'height': 999.78214}, {'x': 1792.69064, 'y': 402.5525, 'width': 617.02734, 'height': 1204.358}]",opacity 1 677.42216 197.97662 1545.21983 1197.75876 opacity 1 1792.69064 402.5525 2409.71798 1606.9105,9d514ce429a7,0,1,0,0
3,001398f4ff4f.dcm,"[{'x': 2729, 'y': 2181.33331, 'width': 948.00012, 'height': 604}]",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,0,0,1
4,001bd15d1891.dcm,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'height': 1106}, {'x': 2578.56661, 'y': 998.66667, 'width': 662.66667, 'height': 1120}]",opacity 1 623.23328 1050 1337.23328 2156 opacity 1 2578.56661 998.66667 3241.23328 2118.66667,dfd9fdd85a3e,0,1,0,0


Criamos um Folder onde iremos extrair todas as imagens classificadas com COVID-19: mkdir Dataset/Covid

In [18]:
dataSetPath = Path('/Users/marcelo/IADD/siim-covid19-detection')
COVID_DIR = os.path.join(dataSetPath, "COVID")
print(COVID_DIR)

if not os.path.exists(COVID_DIR):
   os.mkdir(COVID_DIR)
   print("Folder COVID-19 criado.....")


/Users/marcelo/IADD/siim-covid19-detection/COVID
Folder COVID-19 criado.....


Criamos um Folder onde iremos extrair todas as imagens classificadas com NORMAL: mkdir Dataset/Normal

In [19]:
dataSetPath = Path('/Users/marcelo/IADD/siim-covid19-detection')
NORMAL_DIR = os.path.join(dataSetPath, "NORMAL")
print(NORMAL_DIR)

if not os.path.exists(NORMAL_DIR):
   os.mkdir(NORMAL_DIR)
   print("Folder NORMAL_DIR criado.....")

/Users/marcelo/IADD/siim-covid19-detection/NORMAL
Folder NORMAL_DIR criado.....


Proximo passo é Copiar as imagens Classificadas como Nomal:  Negative for Pneumonia = 1  para o Folder NORMAL  e Negative for Pneumonia = 0 e Typical Appearance = 1 para o Folder COVID

In [17]:
def getRealPath(studyID):
    copyDICOM = 0
    path = f'{dataSetPath}/train/' + studyID

    files = glob.glob(path + '/*' + '/*.dcm')
    
    listPaths = train_images
    
    for f in files:
        print(f)
        shutil.copy2(f, NORMAL_DIR)
        print("Movendo imagem...", cnt)
        copyDICOM += 1
    
    
    

In [20]:
def copyExams(studyID, newPath):
    path = f'{dataSetPath}/train/' + studyID
    files = glob.glob(path + '/*' + '/*.dcm')
         
    for file in files:
        shutil.copy2(file, newPath)
             
    
    

In [21]:
#Base tem 1.676 Estudos Normais com 1736 imagens dcm normais

NORMAL = 1

#trainingPath = os.path.join(dataSetPath,"train")
copyDICOM = 0

for (i,row) in merged.iterrows():
  if row["Negative for Pneumonia"] == NORMAL:
    studyID = row["StudyInstanceUID"]
    copyExams(studyID,NORMAL_DIR)
    copyDICOM += 1
    print("Movendo imagem...", copyDICOM) 
    #folder = row["StudyInstanceUID"]
    #getRealPath(folder)
    

Movendo imagem... 1
Movendo imagem... 2
Movendo imagem... 3
Movendo imagem... 4
Movendo imagem... 5
Movendo imagem... 6
Movendo imagem... 7
Movendo imagem... 8
Movendo imagem... 9
Movendo imagem... 10
Movendo imagem... 11
Movendo imagem... 12
Movendo imagem... 13
Movendo imagem... 14
Movendo imagem... 15
Movendo imagem... 16
Movendo imagem... 17
Movendo imagem... 18
Movendo imagem... 19
Movendo imagem... 20
Movendo imagem... 21
Movendo imagem... 22
Movendo imagem... 23
Movendo imagem... 24
Movendo imagem... 25
Movendo imagem... 26
Movendo imagem... 27
Movendo imagem... 28
Movendo imagem... 29
Movendo imagem... 30
Movendo imagem... 31
Movendo imagem... 32
Movendo imagem... 33
Movendo imagem... 34
Movendo imagem... 35
Movendo imagem... 36
Movendo imagem... 37
Movendo imagem... 38
Movendo imagem... 39
Movendo imagem... 40
Movendo imagem... 41
Movendo imagem... 42
Movendo imagem... 43
Movendo imagem... 44
Movendo imagem... 45
Movendo imagem... 46
Movendo imagem... 47
Movendo imagem... 48
M

In [22]:
#Base tem 3.007 Estudos com Pneumonia e  aparencia Típica para COVID - apos recorte 1.671
COVID = 0
TypicalAppearance = 1

copyDICOM = 0

for (i,row) in merged.iterrows():
  if row["Negative for Pneumonia"] == COVID and row["Typical Appearance"] ==  TypicalAppearance: 
    studyID = row["StudyInstanceUID"]
    copyExams(studyID,COVID_DIR)
    copyDICOM += 1
    print("Movendo imagem...", copyDICOM)

Movendo imagem... 1
Movendo imagem... 2
Movendo imagem... 3
Movendo imagem... 4
Movendo imagem... 5
Movendo imagem... 6
Movendo imagem... 7
Movendo imagem... 8
Movendo imagem... 9
Movendo imagem... 10
Movendo imagem... 11
Movendo imagem... 12
Movendo imagem... 13
Movendo imagem... 14
Movendo imagem... 15
Movendo imagem... 16
Movendo imagem... 17
Movendo imagem... 18
Movendo imagem... 19
Movendo imagem... 20
Movendo imagem... 21
Movendo imagem... 22
Movendo imagem... 23
Movendo imagem... 24
Movendo imagem... 25
Movendo imagem... 26
Movendo imagem... 27
Movendo imagem... 28
Movendo imagem... 29
Movendo imagem... 30
Movendo imagem... 31
Movendo imagem... 32
Movendo imagem... 33
Movendo imagem... 34
Movendo imagem... 35
Movendo imagem... 36
Movendo imagem... 37
Movendo imagem... 38
Movendo imagem... 39
Movendo imagem... 40
Movendo imagem... 41
Movendo imagem... 42
Movendo imagem... 43
Movendo imagem... 44
Movendo imagem... 45
Movendo imagem... 46
Movendo imagem... 47
Movendo imagem... 48
M

** Dividir o DataSet em Treino e Validação **

In [ ]:
#split into training and validation set
import random
import splitfolders 

root_dir = "/Users/marcelo/IADD/Classificacao_COVID_NORMAL"
output_dir = "/Users/marcelo/IADD/Classificacao_COVID_NORMAL/Output"

# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(root_dir, output=output_dir, seed=1337, ratio=(.8, .2), group_prefix=None) # default values

/Users/marcelo/IADD/Classificacao_COVID_NORMAL
